In [1]:
! nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [2]:
! wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2022-10-15 09:34:23--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2022-10-15 09:34:23 (87.9 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [3]:
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

--2022-10-15 09:34:23--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 209.85.146.128, 209.85.147.128, 142.250.125.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|209.85.146.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip’

10_food_classes_10_ 100%[===================>] 160.74M   226MB/s    in 0.7s    

2022-10-15 09:34:24 (226 MB/s) - ‘10_food_classes_10_percent.zip’ saved [168546183/168546183]



In [4]:
from helper_functions import unzip_data , walk_through_dir

In [5]:
unzip_data("10_food_classes_10_percent.zip")

In [6]:
walk_through_dir("./10_food_classes_10_percent/")

There are 2 directories and 0 images in './10_food_classes_10_percent/'.
There are 10 directories and 0 images in './10_food_classes_10_percent/train'.
There are 0 directories and 75 images in './10_food_classes_10_percent/train/ramen'.
There are 0 directories and 75 images in './10_food_classes_10_percent/train/sushi'.
There are 0 directories and 75 images in './10_food_classes_10_percent/train/hamburger'.
There are 0 directories and 75 images in './10_food_classes_10_percent/train/chicken_curry'.
There are 0 directories and 75 images in './10_food_classes_10_percent/train/steak'.
There are 0 directories and 75 images in './10_food_classes_10_percent/train/ice_cream'.
There are 0 directories and 75 images in './10_food_classes_10_percent/train/pizza'.
There are 0 directories and 75 images in './10_food_classes_10_percent/train/grilled_salmon'.
There are 0 directories and 75 images in './10_food_classes_10_percent/train/fried_rice'.
There are 0 directories and 75 images in './10_food_c

In [7]:
train_dir = "./10_food_classes_10_percent/train"
test_dir = "./10_food_classes_10_percent/test"

In [8]:
import tensorflow as tf

In [9]:
IMAGE_SIZE =(224,224)
BATCH_SIZE= 32
train_10_percent = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    image_size =IMAGE_SIZE,
    label_mode="categorical",
    batch_size= BATCH_SIZE
)

test_10_percent = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    image_size =IMAGE_SIZE,
    label_mode="categorical",
    batch_size= BATCH_SIZE
)

Found 750 files belonging to 10 classes.
Found 750 files belonging to 10 classes.


# How to create a Functuonal Model

In [10]:
from helper_functions import create_tensorboard_callback

In [ ]:
# 1- create our baseline basemodel
baseline =tf.keras.applications.EfficientNetB0(include_top=False)
# 2- freeze our model "feature extractor "
baseline.trainable = False
# 3- input layer
inputs = tf.keras.layers.Input( shape = (224,224,3) , name = "Input Layer")
# 4- rescaling layer
#x = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)(inputs)
# 5- pass the inputs to the baseline model
x = baseline(inputs)
print(f"after adding the inputs layers {x.shape}")

# 6- avrage poolong layer to reduce computations
x = tf.keras.layers.GlobalAveragePooling2D( name ="global_avrage_layer")(x)
print(f"after global_pooling layers {x.shape}")

# 7- output layer
outputs = tf.keras.layers.Dense(10, activation="softmax")(x)
# 8- combine the inputs and outputs 
model_0 = tf.keras.Model(inputs, outputs)
# 9- compile the model
model_0.compile(
    loss = "categorical_crossentropy",
    optimizer = tf.keras.optimizers.Adam(),
    metrics = ["accuracy"]
)
# 10- fit the model

history =model_0.fit(train_10_percent,
            epochs = 5,
            steps_per_epoch = len(train_10_percent),
            validation_data = test_10_percent,
            validation_steps = len(test_10_percent),
            callbacks = [create_tensorboard_callback("transfer_learning", "10_percent_data")])

16705208/16705208 [==============================] - 0s 0us/step
after adding the inputs layers (None, 7, 7, 1280)
after global_pooling layers (None, 1280)
Saving TensorBoard log files to: transfer_learning/10_percent_data/20221015-093437
Epoch 1/5
24/24 [==============================] - 139s 6s/step - loss: 1.8639 - accuracy: 0.4280 - val_loss: 1.3057 - val_accuracy: 0.6960
Epoch 2/5
24/24 [==============================] - 128s 5s/step - loss: 1.1087 - accuracy: 0.7533 - val_loss: 0.8566 - val_accuracy: 0.8293
Epoch 3/5
24/24 [==============================] - 128s 5s/step - loss: 0.8013 - accuracy: 0.8413 - val_loss: 0.6602 - val_accuracy: 0.8800
Epoch 4/5
24/24 [==============================] - 128s 5s/step - loss: 0.6432 - accuracy: 0.8680 - val_loss: 0.5472 - val_accuracy: 0.8960
Epoch 5/5
24/24 [==============================] - 128s 5s/step - loss: 0.5457 - accuracy: 0.8880 - val_loss: 0.4704 - val_accuracy: 0.9173


In [ ]:
model_0.evaluate(test_10_percent)

24/24 [==============================] - 46s 2s/step - loss: 0.4704 - accuracy: 0.9173


[0.4703560471534729, 0.9173333048820496]

## getting a feature vector from trained model

In [ ]:
tensor = tf.random.normal(shape = (1,4,4,3))

In [ ]:
global_tensot = tf.keras.layers.GlobalAveragePooling2D() (tensor)

In [ ]:

# Define input tensor shape (same number of dimensions as the output of efficientnetb0)
input_shape = (1, 4, 4, 3)

# Create a random tensor
tf.random.set_seed(42)
input_tensor = tf.random.normal(input_shape)
print(f"Random input tensor:\n {input_tensor}\n")

# Pass the random tensor through a global average pooling 2D layer
global_average_pooled_tensor = tf.keras.layers.GlobalAveragePooling2D()(input_tensor)
print(f"2D global average pooled random tensor:\n {global_average_pooled_tensor}\n")

# Check the shapes of the different tensors
print(f"Shape of input tensor: {input_tensor.shape}")
print(f"Shape of 2D global averaged pooled input tensor: {global_average_pooled_tensor.shape}")

Random input tensor:
 [[[[ 0.3274685  -0.8426258   0.3194337 ]
   [-1.4075519  -2.3880599  -1.0392479 ]
   [-0.5573232   0.539707    1.6994323 ]
   [ 0.28893656 -1.5066116  -0.26454744]]

  [[-0.59722406 -1.9171132  -0.62044144]
   [ 0.8504023  -0.40604794 -3.0258412 ]
   [ 0.9058464   0.29855987 -0.22561555]
   [-0.7616443  -1.891714   -0.9384712 ]]

  [[ 0.77852213 -0.47338897  0.97772694]
   [ 0.24694404  0.20573747 -0.5256233 ]
   [ 0.32410017  0.02545409 -0.10638497]
   [-0.6369475   1.1603122   0.2507359 ]]

  [[-0.41728497  0.40125778 -1.4145442 ]
   [-0.59318566 -1.6617213   0.33567193]
   [ 0.10815629  0.2347968  -0.56668764]
   [-0.35819843  0.88698626  0.5274477 ]]]]

2D global average pooled random tensor:
 [[-0.09368646 -0.45840445 -0.28855976]]

Shape of input tensor: (1, 4, 4, 3)
Shape of 2D global averaged pooled input tensor: (1, 3)


In [ ]:
INPUT_SHAPE = (1,4,4,3)
tensor = tf.random.normal(INPUT_SHAPE)
print("the tensor before global max pool {}".format(tensor))

tensor_global = tf.keras.layers.GlobalMaxPool2D() (tensor)
print(tensor.shape)
print(tensor_global.shape)






the tensor before global max pool [[[[ 8.4224582e-02 -8.6090374e-01  3.7812304e-01]
   [-5.1962738e-03 -4.9453196e-01  6.1781919e-01]
   [-3.3082047e-01 -1.3840806e-03 -4.2373410e-01]
   [-1.3872087e+00 -1.5488191e+00 -5.3198391e-01]]

  [[-4.4756433e-01 -2.0115814e+00 -5.7926011e-01]
   [ 5.7938927e-01  1.3041967e+00  6.7720258e-01]
   [-7.4587613e-01  1.0378964e+00  1.3820479e+00]
   [ 1.4319171e+00 -3.7643117e-01  9.8158473e-01]]

  [[-2.3597862e-01 -3.3763260e-01 -8.9593250e-01]
   [ 4.2754072e-01 -3.8105518e-01  4.7006992e-01]
   [ 3.5413779e-02 -2.9272759e+00 -9.6707004e-01]
   [-4.1402709e-01 -4.0137586e-01  6.2328768e-01]]

  [[-9.3648863e-01  9.5449388e-01  4.9025390e-01]
   [-9.9804842e-01 -1.1686406e+00 -6.7897290e-01]
   [ 1.7331039e+00  7.8643018e-01  9.2237018e-02]
   [ 2.2711790e-01 -9.1896117e-02  1.1224977e+00]]]]
(1, 4, 4, 3)
(1, 3)


In [ ]:
tensor_global

<tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[1.7331039, 1.3041967, 1.3820479]], dtype=float32)>

In [ ]:
tf.reduce_max(tensor, axis=(1,2))

<tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[1.7331039, 1.3041967, 1.3820479]], dtype=float32)>

# Model_1

In [ ]:
! wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_1_percent.zip

--2022-10-15 09:46:40--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_1_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.125.128, 142.250.136.128, 142.250.148.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.125.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 133612354 (127M) [application/zip]
Saving to: ‘10_food_classes_1_percent.zip’

10_food_classes_1_p 100%[===================>] 127.42M  79.2MB/s    in 1.6s    

2022-10-15 09:46:41 (79.2 MB/s) - ‘10_food_classes_1_percent.zip’ saved [133612354/133612354]



In [ ]:
unzip_data("10_food_classes_1_percent.zip")

In [ ]:
walk_through_dir("10_food_classes_1_percent")

There are 2 directories and 0 images in '10_food_classes_1_percent'.
There are 10 directories and 0 images in '10_food_classes_1_percent/train'.
There are 0 directories and 7 images in '10_food_classes_1_percent/train/ramen'.
There are 0 directories and 7 images in '10_food_classes_1_percent/train/sushi'.
There are 0 directories and 7 images in '10_food_classes_1_percent/train/hamburger'.
There are 0 directories and 7 images in '10_food_classes_1_percent/train/chicken_curry'.
There are 0 directories and 7 images in '10_food_classes_1_percent/train/steak'.
There are 0 directories and 7 images in '10_food_classes_1_percent/train/ice_cream'.
There are 0 directories and 7 images in '10_food_classes_1_percent/train/pizza'.
There are 0 directories and 7 images in '10_food_classes_1_percent/train/grilled_salmon'.
There are 0 directories and 7 images in '10_food_classes_1_percent/train/fried_rice'.
There are 0 directories and 7 images in '10_food_classes_1_percent/train/chicken_wings'.
There a

### getting and preroceessing data

In [ ]:
train_1_data="10_food_classes_1_percent/train"
test_1_data ="10_food_classes_1_percent/test"

In [ ]:
train_1_percent = tf.keras.preprocessing.image_dataset_from_directory( train_1_data,
                                                                      image_size= IMAGE_SIZE,
                                                                      batch_size = 32,
                                                                      label_mode= "categorical",
                                                                      )
test_1_percent = tf.keras.preprocessing.image_dataset_from_directory(test_1_data,
                                                                      image_size= IMAGE_SIZE,
                                                                      batch_size = 32,
                                                                      label_mode= "categorical",
                                                                      )

Found 70 files belonging to 10 classes.
Found 2500 files belonging to 10 classes.


### creating the model_1

In [ ]:
from tensorflow.keras import preprocessing
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
# data augmantation layer
data_augment = keras.Sequential([
    layers.RandomFlip(),
    layers.RandomWidth(0.2),
    layers.RandomBrightness(0.2),
    layers.RandomZoom(0.2)
 ],name = "data_augmantation")

In [ ]:
 # visualisation

In [ ]:
#  creating model_1
# 1- creating baseline
baseline = tf.keras.applications.EfficientNetB0(include_top=False)
# 2- freeze model
baseline.trainable=False
# 3- create input layer
inputs = tf.keras.layers.Input(shape = (224,224,3))
# 4- create data augmantion layer
x = data_augment(inputs)
# 5- adding baseline
x = baseline(x, training=False)
# 6- vector feature layer
x = tf.keras.layers.GlobalAveragePooling2D()(x)
# 7- outpu layer
outputs = tf.keras.layers.Dense(10, activation= "softmax") (x)
# 8- making the model
model_1 = tf.keras.Model(inputs, outputs)


In [ ]:
# compile
model_1.compile(loss = "categorical_crossentropy",
                optimizer = "Adam",
                metrics = "accuracy")

In [ ]:
history_1 =model_1.fit(train_1_percent,
            epochs = 5,
            steps_per_epoch = len(train_1_percent),
            validation_data = test_1_percent,
            validation_steps = len(test_1_percent),
            callbacks = [create_tensorboard_callback("transfer_learning", "1_percent_data")])

In [ ]:
model_1.evaluate(test_10_percent)

In [ ]:
model_1.summary()

In [ ]:
from helper_functions import plot_loss_curves

In [ ]:
plot_loss_curves(history_1)

## Buliding model_2

In [ ]:
model_2 = model_1

In [ ]:
# compile
model_2.compile(loss = "categorical_crossentropy",
                optimizer = "Adam",
                metrics = "accuracy")

In [ ]:
history_2 =model_2.fit(train_10_percent,
            epochs = 5,
            steps_per_epoch = len(train_10_percent),
            validation_data = test_10_percent,
            validation_steps = len(test_10_percent),
            callbacks = [create_tensorboard_callback("transfer_learning", "10_percent_data_model2")])

### creating a check points callback

In [ ]:
checkPoint = tf.keras.callbacks.ModelCheckpoint("10_percent_weights/checkpoint.ckpt",
                                                verpose = 1,
                                                save_only_weights= True)

In [ ]:
history_2 =model_2.fit(train_10_percent,
            epochs = 5,
            steps_per_epoch = len(train_10_percent),
            validation_data = test_10_percent,
            validation_steps = len(test_10_percent),
            callbacks = [checkPoint,create_tensorboard_callback("transfer_learning", "10_percent_data_model2")])

In [ ]:
model_0.evaluate(test_10_percent), model_2.evaluate(test_10_percent)

In [ ]:
plot_loss_curves(history_2)

In [ ]:
model_2.load_weights("10_percent_weights/checkpoint.ckpt")

In [ ]:
model_2.evaluate(test_10_percent)

In [ ]:
# for fine tunning make the last 10 layers are trainable
baseline.trainable = True
for layer in baseline.layers[:-10]:
  layer.trainable = False

In [ ]:
# compile the modele again and reduce the lr by 10x not to cause overfitting
model_2.compile(
    loss = "categorical_crossentropy",
    optimizer = tf.keras.optimizers.Adam(lr=0.0001),
    metrics = "accuracy"
)

In [ ]:
for i, layer in enumerate( model_2.layers[2].layers):
  print(i, layer.name, layer.trainable)